In [1]:
!pip install mediapipe opencv-python numpy

In [3]:
import cv2
import mediapipe as mp
import numpy as np
import math
from enum import Enum
import time

In [5]:
# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(min_detection_confidence=0.6, min_tracking_confidence=0.5)

I0000 00:00:1755884971.466193 1371499 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M4


In [7]:
def debug_print_landmarks(landmarks):
    """
    Print all relevant landmark positions for debugging
    """
    print("\n--- DEBUG: Landmark Positions ---")

    # Define key landmarks for this exercise
    key_points = {
        'LEFT_HIP': mp_pose.PoseLandmark.LEFT_HIP.value,
        'RIGHT_HIP': mp_pose.PoseLandmark.RIGHT_HIP.value,
        'LEFT_KNEE': mp_pose.PoseLandmark.LEFT_KNEE.value,
        'RIGHT_KNEE': mp_pose.PoseLandmark.RIGHT_KNEE.value,
        'LEFT_ANKLE': mp_pose.PoseLandmark.LEFT_ANKLE.value,
        'RIGHT_ANKLE': mp_pose.PoseLandmark.RIGHT_ANKLE.value,
        'LEFT_HEEL': mp_pose.PoseLandmark.LEFT_HEEL.value,
        'RIGHT_HEEL': mp_pose.PoseLandmark.RIGHT_HEEL.value,
        'LEFT_FOOT_INDEX': mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value,
        'RIGHT_FOOT_INDEX': mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value,
        'LEFT_SHOULDER': mp_pose.PoseLandmark.LEFT_SHOULDER.value,
        'RIGHT_SHOULDER': mp_pose.PoseLandmark.RIGHT_SHOULDER.value,
        'LEFT_WRIST': mp_pose.PoseLandmark.LEFT_WRIST.value,
        'RIGHT_WRIST': mp_pose.PoseLandmark.RIGHT_WRIST.value,
        'LEFT_INDEX': mp_pose.PoseLandmark.LEFT_INDEX.value,
        'RIGHT_INDEX': mp_pose.PoseLandmark.RIGHT_INDEX.value,
    }

    for name, idx in key_points.items():
        landmark = landmarks[idx]
        print(f"{name}: x={landmark.x:.3f}, y={landmark.y:.3f}, z={landmark.z:.3f}, visibility={landmark.visibility:.3f}")

    # Calculate and print some useful measurements
    print("\n--- Calculated Values ---")

    # Hip height average
    left_hip_y = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y
    right_hip_y = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y
    avg_hip_y = (left_hip_y + right_hip_y) / 2
    print(f"Average Hip Height (Y): {avg_hip_y:.3f}")

    # Knee height average
    left_knee_y = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y
    right_knee_y = landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y
    avg_knee_y = (left_knee_y + right_knee_y) / 2
    print(f"Average Knee Height (Y): {avg_knee_y:.3f}")

    # Ankle positions
    left_ankle_y = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y
    right_ankle_y = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y
    print(f"Left Ankle Y: {left_ankle_y:.3f}, Right Ankle Y: {right_ankle_y:.3f}")

    # Hip to knee distance (normalized)
    hip_knee_diff = avg_hip_y - avg_knee_y
    print(f"Hip-Knee Height Difference: {hip_knee_diff:.3f}")

    print("-" * 40)

In [9]:
# Define states for the flexibility test
class TestState(Enum):
    SITTING_CHECK = 1
    FOOT_FLAT_CHECK = 2
    LEG_EXTENDED_CHECK = 3
    HANDS_POSITION_CHECK = 4
    INHALE_PREPARATION = 5
    FORWARD_REACH = 6
    HOLD_POSITION = 7
    MEASUREMENT = 8
    COMPLETE = 9

class FlexibilityTest:
    def __init__(self):
        self.current_state = TestState.SITTING_CHECK
        self.state_passed = False
        self.hold_timer = 0
        self.hold_start_time = None
        self.measurement_result = None
        self.debug_enabled = False
        self.advance = False

    def get_state_message(self):
        """Get instruction message for current state"""
        messages = {
            TestState.SITTING_CHECK: "Step 1: Please sit on the edge of a chair",
            TestState.FOOT_FLAT_CHECK: "Step 2: Place one foot flat on the floor",
            TestState.LEG_EXTENDED_CHECK: "Step 3: Extend other leg forward - knee straight, heel on floor, ankle at 90°",
            TestState.HANDS_POSITION_CHECK: "Step 4: Place one hand on top of the other with middle fingers even",
            TestState.INHALE_PREPARATION: "Step 5: Take a deep breath in",
            TestState.FORWARD_REACH: "Step 6: Exhale and reach forward toward your toes",
            TestState.HOLD_POSITION: "Step 7: Hold the position for 2 seconds",
            TestState.MEASUREMENT: "Measuring flexibility...",
            TestState.COMPLETE: f"Test complete! Result: {self.measurement_result}"
        }
        return messages.get(self.current_state, "")

    def advance_state(self):
        """Move to the next state"""
        if self.current_state == TestState.SITTING_CHECK:
            self.current_state = TestState.FOOT_FLAT_CHECK
        elif self.current_state == TestState.FOOT_FLAT_CHECK:
            self.current_state = TestState.LEG_EXTENDED_CHECK
        elif self.current_state == TestState.LEG_EXTENDED_CHECK:
            self.current_state = TestState.HANDS_POSITION_CHECK
        elif self.current_state == TestState.HANDS_POSITION_CHECK:
            self.current_state = TestState.INHALE_PREPARATION
        elif self.current_state == TestState.INHALE_PREPARATION:
            self.current_state = TestState.FORWARD_REACH
        elif self.current_state == TestState.FORWARD_REACH:
            self.current_state = TestState.HOLD_POSITION
            self.hold_start_time = cv2.getTickCount()
        elif self.current_state == TestState.HOLD_POSITION:
            self.current_state = TestState.MEASUREMENT
        elif self.current_state == TestState.MEASUREMENT:
            self.current_state = TestState.COMPLETE

    def reset_test(self):
        """Reset to beginning"""
        self.current_state = TestState.SITTING_CHECK
        self.state_passed = False
        self.hold_timer = 0
        self.hold_start_time = None
        self.measurement_result = None

In [11]:
# State check functions
def check_sitting_position(landmarks):
    """Check if person is sitting based on hip and knee angles"""
    left_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]
    right_hip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]
    left_knee = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value]
    right_knee = landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value]

    avg_hip_y = (left_hip.y + right_hip.y) / 2
    avg_knee_y = (left_knee.y + right_knee.y) / 2
    hip_knee_distance = abs(avg_hip_y - avg_knee_y)

    if hip_knee_distance < 0.15:
        return True, "Sitting position detected :)"
    else:
        return False, "Not sitting - please sit on edge of chair"

In [13]:
def check_foot_flat_on_floor(landmarks):
    """
    Checks if at least one foot is flat by comparing heel and foot index Y positions.
    """
    Y_TOL = 0.06  # tolerance for vertical alignment

    def foot_flat(heel, toe):
        return abs(heel.y - toe.y) < Y_TOL

    # Left foot
    left_flat = foot_flat(
        landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value],
        landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value]
    )

    # Right foot
    right_flat = foot_flat(
        landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value],
        landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value]
    )

    if left_flat or right_flat:
        return True, "Foot is flat on the floor :)"
    else:
        return False, "Please place one foot flat on the floor"

In [15]:
def calculate_angle_3d(a, b, c):
    """
    Calculates the angle at point b in 3D given points a, b, c.
    Each point is an object with .x, .y, .z attributes.
    """
    # Vectors BA and BC
    ba = (a.x - b.x, a.y - b.y, a.z - b.z)
    bc = (c.x - b.x, c.y - b.y, c.z - b.z)

    # Dot product and magnitudes
    dot_product = ba[0]*bc[0] + ba[1]*bc[1] + ba[2]*bc[2]
    mag_ba = math.sqrt(ba[0]**2 + ba[1]**2 + ba[2]**2)
    mag_bc = math.sqrt(bc[0]**2 + bc[1]**2 + bc[2]**2)

    if mag_ba == 0 or mag_bc == 0:
        return 0

    angle_rad = math.acos(max(min(dot_product / (mag_ba * mag_bc), 1.0), -1.0))
    return math.degrees(angle_rad)

def check_leg_extended(landmarks):
    """
    Checks if either leg is straight using 3D hip-knee-ankle angle.
    """
    lh, lk, la = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value], landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value], landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value]
    rh, rk, ra = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value], landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value], landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value]

    left_knee_angle = calculate_angle_3d(lh, lk, la)
    right_knee_angle = calculate_angle_3d(rh, rk, ra)

    ANGLE_THRESHOLD = 80  # degrees

    if left_knee_angle > ANGLE_THRESHOLD:
        return True, f"Left leg is straight ({left_knee_angle:.1f}°) :)"
    elif right_knee_angle > ANGLE_THRESHOLD:
        return True, f"Right leg is straight ({right_knee_angle:.1f}°) :)"
    else:
        return False, f"Please extend one leg straight (L:{left_knee_angle:.1f}, R:{right_knee_angle:.1f})"

In [17]:
def hand_center(hand_landmarks):
    """Calculate 3D center of wrist, index, and pinky"""
    x = (hand_landmarks[0].x + hand_landmarks[1].x + hand_landmarks[2].x) / 3
    y = (hand_landmarks[0].y + hand_landmarks[1].y + hand_landmarks[2].y) / 3
    z = (hand_landmarks[0].z + hand_landmarks[1].z + hand_landmarks[2].z) / 3
    return type('Point', (object,), {'x': x, 'y': y, 'z': z})()

def check_hands_position(landmarks):
    """
    Checks if one hand is on top of the other by comparing the center points
    of wrist, index, and pinky.
    """
    # Left hand landmarks
    left_landmarks = [
        landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value],
        landmarks[mp_pose.PoseLandmark.LEFT_INDEX.value],
        landmarks[mp_pose.PoseLandmark.LEFT_PINKY.value]
    ]
    
    # Right hand landmarks
    right_landmarks = [
        landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value],
        landmarks[mp_pose.PoseLandmark.RIGHT_INDEX.value],
        landmarks[mp_pose.PoseLandmark.RIGHT_PINKY.value]
    ]
    
    # Calculate hand centers
    left_center = hand_center(left_landmarks)
    right_center = hand_center(right_landmarks)
    
    # Distance between centers
    dist = math.sqrt((left_center.x - right_center.x)**2 +
                     (left_center.y - right_center.y)**2 +
                     (left_center.z - right_center.z)**2)
    
    # Threshold for "hands together"
    DIST_THRESHOLD = 0.06
    
    if dist < DIST_THRESHOLD:
        return True, f"Hands positioned correctly :) (distance: {dist:.3f})"
    else:
        return False, f"Place one hand on top of the other (distance: {dist:.3f})"

In [19]:
def check_inhale(landmarks):
    """
    Instructs the subject to inhale in preparation for forward reach.
    Always passes after showing the instruction.
    """
    return True, "Inhale deeply... next, as you exhale, you'll reach forward."

In [21]:
def check_forward_reach(landmarks):
    # TODO: Implement
    return False, "Reach forward"

In [23]:
import os
os.getcwd()


'/Users/chandini'

In [27]:
import os
os.getcwd()


'/Users/chandini'